In [17]:
with open("output11.txt") as file:
    corpus = file.read()

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

In [19]:
sentences = sent_tokenize(corpus)

In [20]:
document = [{'sentence': x, 'index' : i} for i, x in enumerate(sentences)]

Combining the sentences i to i+1 and i-1 to keep context intact.

In [21]:
def combine_sentences(sentences, buffer_size=1):
    # Go through each sentence dict
    for i in range(len(sentences)):

        # Create a string that will hold the sentences which are joined
        combined_sentence = ''

        # Add sentences before the current one, based on the buffer size.
        for j in range(i - buffer_size, i):
            # Check if the index j is not negative (to avoid index out of range like on the first one)
            if j >= 0:
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += sentences[j]['sentence'] + ' '

        # Add the current sentence
        combined_sentence += sentences[i]['sentence']

        # Add sentences after the current one, based on the buffer size
        for j in range(i + 1, i + 1 + buffer_size):
            # Check if the index j is within the range of the sentences list
            if j < len(sentences):
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += ' ' + sentences[j]['sentence']

        # Then add the whole thing to your dict
        # Store the combined sentence in the current sentence dict
        sentences[i]['combined_sentence'] = combined_sentence

    return sentences

In [22]:
sentences = combine_sentences(document)

In [23]:
len(sentences)

1358957

In [24]:
com_sentences = []

for x in sentences:
    com_sentences.append(x['combined_sentence'])

In [25]:
len(com_sentences)

1358957

In [26]:
import torch

In [ ]:
from sentence_transformers import SentenceTransformer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer("all-MiniLM-L6-v2")
model.to(device)


In [28]:
import time
start_time = time.time()
embeddings = model.encode(com_sentences,device=device)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for encoding: {elapsed_time} seconds")

Time taken for encoding: 3635.365577697754 seconds


In [29]:
import json
output_file = "embeddings.jsonl"
with open(output_file, 'w') as f:
    for sentence, embedding in zip(com_sentences, embeddings):
        data = {
            "sentence": sentence,
            "embedding": embedding.tolist()  # Convert embedding tensor to list for JSON serialization
        }
        f.write(json.dumps(data) + '\n')